In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cuda


In [3]:
os.chdir("C:\\Users\\GL75\\OneDrive\\桌面\\深度學習\\Lab3")

In [4]:
def getData(mode):
    if mode == 'train':
        img = pd.read_csv('train_img.csv')
        label = pd.read_csv('train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('test_img.csv')
        label = pd.read_csv('test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)

In [5]:
batch_size18 = 32 # GPU maximum memory for my laptop
batch_size50 = 8

In [6]:
mean = torch.load(os.path.join("C:\\Users\\GL75\\OneDrive\\桌面\\深度學習\\Lab3", 'train_mean.pt'))
std = torch.load(os.path.join("C:\\Users\\GL75\\OneDrive\\桌面\\深度學習\\Lab3", 'train_std.pt'))

In [7]:
'''for training data'''
print(mean) # tensor([0.3749, 0.2602, 0.1857])
print(std) # tensor([0.2526, 0.1780, 0.1291])

tensor([0.3749, 0.2602, 0.1857])
tensor([0.2526, 0.1780, 0.1291])


In [8]:
class RetinopathyDataset(Dataset):
    def __init__(self, root, mode, mean, std):
        self.root = root
        self.mode = mode
        self.img_names, self.labels = getData(self.mode)
        self.length = len(self.img_names)
        self.transformations = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
        print("> Found %d images..." % (self.length))
    def __len__(self):
        return self.length
    def __getitem__(self, index):
        img_name = os.path.join(self.root, self.img_names[index] + '.jpeg')
        img = Image.open(img_name)
        img = self.transformations(img)
        label = self.labels[index]
        return img, label

In [9]:
class ResNet18(nn.Module):
    def __init__(self, num_classes = 5, pretrained = True):
        super().__init__()
        self.model = models.resnet18(pretrained = pretrained)
        if pretrained:
            for param in self.model.parameters():
                param.require_grads = False
        num_of_fc_input_neurons = self.model.fc.in_features # 512
        self.model.fc = nn.Linear(num_of_fc_input_neurons, num_classes)
    def forward(self, X):
        outputs = self.model(X)
        return outputs

In [10]:
class ResNet50(nn.Module):
    def __init__(self, num_classes = 5, pretrained = True):
        super().__init__()
        self.model = models.resnet50(pretrained = pretrained)
        if pretrained:
            for param in self.model.parameters():
                param.require_grads = False
        num_of_fc_input_neurons = self.model.fc.in_features # 512
        self.model.fc = nn.Linear(num_of_fc_input_neurons, num_classes)
    def forward(self, X):
        outputs = self.model(X)
        return outputs

In [11]:
def test(model, test_dataloader, device):
    model.eval()
    with torch.no_grad():
        accuracy = 0.0
        for X_batched, y_batched in test_dataloader:
            X_batched = X_batched.to(device, dtype = torch.float)
            y_batched = y_batched.to(device, dtype = torch.long)
            predictions = model(X_batched)
            accuracy += predictions.max(dim = 1)[1].eq(y_batched).sum().item()
        accuracy = accuracy / len(test_dataloader.dataset) * 100
        
    return accuracy

In [12]:
test_dataset = RetinopathyDataset('D:/DL_lab3_data', 'test', mean, std)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size18, shuffle = False)
# test_dataloader = DataLoader(test_dataset, batch_size = batch_size50, shuffle = False)

> Found 7025 images...


In [13]:
model = ResNet18(pretrained = True)
model.load_state_dict(torch.load(os.path.join("D:/DL_lab3_params", "ResNet18_with_pretrained.pt"), map_location = device))
# model = ResNet50(pretrained = True)
# model.load_state_dict(torch.load(os.path.join("D:/DL_lab3_params", "ResNet50_with_pretrained.pt"), map_location = device))
model.to(device)
accuracy = test(model, test_dataloader, device)

In [14]:
print("the best testing accuracy: {}".format(accuracy))

the best testing accuracy: 82.06405693950178


In [16]:
df_18 = pd.DataFrame({"epoch": list(range(1, 16)), 
                      "test_acc_ResNet18_with_pretrained": np.load(os.path.join("D:/DL_lab3_params", "test_acc_ResNet18_with_pretrained.npy"))
                     })
# df_50 = pd.DataFrame({"epoch": list(range(1, 11)), 
#                       "test_acc_ResNet50_with_pretrained": np.load(os.path.join("D:/DL_lab3_params", "test_acc_ResNet50_with_pretrained.npy"))
#                      })

In [17]:
for curve_name in df_18.columns[1: ]:
    print("The best {}ing accuracy of ResNet18 model {} pretrained weights is: {:.2f}%".format(curve_name.split("_")[0], curve_name.split("_")[-2], df_18[curve_name].max()))
# for curve_name in df_50.columns[1: ]:
#     print("The best {}ing accuracy of ResNet50 model {} pretrained weights is: {:.2f}%".format(curve_name.split("_")[0], curve_name.split("_")[-2], df_50[curve_name].max()))

The best testing accuracy of ResNet18 model with pretrained weights is: 82.42%
